In [2]:
import requests
# from config import noaa_token

In [4]:
# Austin, TX
latitude = 30.2672
longitude = -97.7431

In [5]:
lat_extent = .03
lng_extent = .03

headers = {'token':noaa_token}
url = f'https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?extent={latitude-lat_extent},{longitude-lng_extent},{latitude+lat_extent},{longitude+lng_extent}'
response = requests.get(url, headers=headers)
response.json()

{'metadata': {'resultset': {'offset': 1, 'count': 5, 'limit': 25}},
 'results': [{'elevation': 175.9,
   'mindate': '2010-04-01',
   'maxdate': '2010-09-08',
   'latitude': 30.257333,
   'name': 'AUSTIN 3.5 SSW, TX US',
   'datacoverage': 0.998,
   'id': 'GHCND:US1TXTV0119',
   'elevationUnit': 'METERS',
   'longitude': -97.768978},
  {'elevation': 178.6,
   'mindate': '2012-10-01',
   'maxdate': '2018-03-07',
   'latitude': 30.254894,
   'name': 'AUSTIN 3.7 SSW, TX US',
   'datacoverage': 0.9288,
   'id': 'GHCND:US1TXTV0171',
   'elevationUnit': 'METERS',
   'longitude': -97.7695789},
  {'elevation': 139.6,
   'mindate': '2013-07-01',
   'maxdate': '2019-04-24',
   'latitude': 30.251644,
   'name': 'AUSTIN 4.0 SSE, TX US',
   'datacoverage': 0.9994,
   'id': 'GHCND:US1TXTV0198',
   'elevationUnit': 'METERS',
   'longitude': -97.726151},
  {'elevation': 183.8,
   'mindate': '2020-01-16',
   'maxdate': '2021-02-23',
   'latitude': 30.237363,
   'name': 'AUSTIN 4.8 S, TX US',
   'datacov

In [6]:
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CITY&limit=1000&offset=1000'
response = requests.get(url, headers=headers)
[x for x in response.json()['results'] if 'austin' in x['name'].lower()]

[{'mindate': '1937-04-01',
  'maxdate': '2021-05-08',
  'name': 'Austin, MN US',
  'datacoverage': 1,
  'id': 'CITY:US270001'},
 {'mindate': '1893-01-01',
  'maxdate': '2021-05-08',
  'name': 'Austin, TX US',
  'datacoverage': 1,
  'id': 'CITY:US480005'}]

In [7]:
url

'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CITY&limit=1000&offset=1000'

In [21]:
#url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations'
data = []
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMIN&datatypeid=TMAX&locationid=CITY:US480005&startdate=2020-01-01&enddate=2020-12-31&limit=1000'
response = requests.get(url, headers=headers)
data.append(response.json()['results'])
for offset in range(1001,response.json()['metadata']['resultset']['count'],1000):
    response = requests.get(f'{url}&offset={offset}', headers=headers)
    data.append(response.json()['results'])
data

3

In [22]:
data = [item for sublist in data for item in sublist]
len(data)

2136

In [2]:
import pandas as pd

In [26]:
df = pd.DataFrame(data)
df['value'] = 9/5 * (df['value'] * 0.1) + 32
df['date'] = pd.to_datetime(df['date'])
df.groupby(['date','datatype'])['value'].mean().reset_index()

,date,datatype,value
0,2020-01-01,TMAX,58.64
1,2020-01-01,TMIN,45.32
2,2020-01-02,TMAX,58.70
3,2020-01-02,TMIN,45.98
4,2020-01-03,TMAX,63.02
...,...,...,...
727,2020-12-29,TMIN,61.64
728,2020-12-30,TMAX,73.64
729,2020-12-30,TMIN,50.00
730,2020-12-31,TMAX,54.02


In [27]:
df = df.groupby(['date','datatype'])['value'].mean().reset_index().pivot(index='date', columns='datatype', values='value')
df.columns.name = ''
df['AVG'] = 0.5 * (df['TMAX'] + df['TMIN'])
df

,TMAX,TMIN,AVG
date,,,
2020-01-01,58.64,45.32,51.98
2020-01-02,58.70,45.98,52.34
2020-01-03,63.02,44.96,53.99
2020-01-04,69.68,39.38,54.53
2020-01-05,73.34,40.34,56.84
...,...,...,...
2020-12-27,74.36,39.98,57.17
2020-12-28,75.68,49.94,62.81
2020-12-29,75.68,61.64,68.66


In [30]:
df.to_json('austin_2020.json', orient='records')

## Grabbing weather data city by city

In [3]:
# Using the Visual Crossing Weather API
url1 = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Austin%2C%20TX/2021-1-1/2021-4-30?unitGroup=us&key=2LN2Z5DVYWXYH9JJXE95C2FU6&include=obs" 
response1 = requests.get(url1)

In [6]:
url2 = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Austin%2C%20TX/2015-7-1/2015-12-31?unitGroup=us&key=2LN2Z5DVYWXYH9JJXE95C2FU6&include=obs"
response2 = requests.get(url2)

In [7]:
response1.json()

{'queryCost': 181,
 'latitude': 30.2676,
 'longitude': -97.743,
 'resolvedAddress': 'Austin, TX, United States',
 'address': 'Austin, TX',
 'timezone': 'America/Chicago',
 'tzoffset': -6.0,
 'days': [{'datetime': '2015-01-01',
   'datetimeEpoch': 1420092000,
   'tempmax': 38.9,
   'tempmin': 34.3,
   'temp': 37.2,
   'feelslikemax': 35.6,
   'feelslikemin': 29.2,
   'feelslike': 32.1,
   'dew': 34.7,
   'humidity': 91.04,
   'precip': 0.48,
   'precipprob': None,
   'precipcover': 70.83,
   'preciptype': None,
   'snow': 0.0,
   'snowdepth': 0.0,
   'windgust': None,
   'windspeed': 8.9,
   'winddir': 125.3,
   'pressure': 1024.9,
   'cloudcover': 100.0,
   'visibility': 4.2,
   'solarradiation': None,
   'solarenergy': None,
   'uvindex': 0.0,
   'sunrise': '07:27:35',
   'sunriseEpoch': 1420118855,
   'sunset': '17:41:36',
   'sunsetEpoch': 1420155696,
   'moonphase': 0.43,
   'conditions': 'Rain, Overcast',
   'description': 'Cloudy skies throughout the day with a chance of rain thr

In [8]:
response2.json()

{'queryCost': 184,
 'latitude': 30.2676,
 'longitude': -97.743,
 'resolvedAddress': 'Austin, TX, United States',
 'address': 'Austin, TX',
 'timezone': 'America/Chicago',
 'tzoffset': -5.0,
 'days': [{'datetime': '2015-07-01',
   'datetimeEpoch': 1435726800,
   'tempmax': 88.8,
   'tempmin': 69.8,
   'temp': 77.3,
   'feelslikemax': 95.9,
   'feelslikemin': 69.8,
   'feelslike': 79.2,
   'dew': 71.3,
   'humidity': 82.62,
   'precip': 0.0,
   'precipprob': None,
   'precipcover': 4.17,
   'preciptype': None,
   'snow': 0.0,
   'snowdepth': 0.0,
   'windgust': 26.1,
   'windspeed': 9.6,
   'winddir': 157.2,
   'pressure': 1016.1,
   'cloudcover': 26.5,
   'visibility': 9.9,
   'solarradiation': None,
   'solarenergy': None,
   'uvindex': 0.0,
   'sunrise': '06:32:48',
   'sunriseEpoch': 1435750368,
   'sunset': '20:36:43',
   'sunsetEpoch': 1435801003,
   'moonphase': 0.5,
   'conditions': 'Partially cloudy',
   'description': 'Partly cloudy throughout the day.',
   'icon': 'partly-clou

In [9]:
austin_21 = response1.json()["days"] + response2.json()["days"]
len(austin_15)

365

In [4]:
austin_21 = response1.json()["days"]
len(austin_21)

120

In [5]:
import json

In [6]:
with open("austin_21.json", "w") as f:
    json.dump(austin_21, f)

In [27]:
# Using the Visual Crossing Weather API
url3 = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Chicago%2C%20IL/2011-1-1/2011-6-30?unitGroup=us&key=2LN2Z5DVYWXYH9JJXE95C2FU6&include=obs" 
response3 = requests.get(url3)

# Using the Visual Crossing Weather API
url4 = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Chicago%2C%20IL/2011-7-1/2011-12-31?unitGroup=us&key=2LN2Z5DVYWXYH9JJXE95C2FU6&include=obs" 
response4 = requests.get(url4)

chicago_11 = response3.json()["days"] + response4.json()["days"]

with open("chicago_11.json", "w") as f:
    json.dump(chicago_11, f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [28]:
response3

<Response [400]>